# LBM Step 2: Registration

## Registration: Correct for rigid/non-rigid movement

- Apply the nonrigid motion correction (NoRMCorre) algorithm for motion correction.
- View pre/most correction movie
- Use quality metrics to evaluate registration quality

In [1]:
import sys
from pathlib import Path
import os
import numpy as np
import zarr

import logging
import mesmerize_core as mc
import matplotlib.pyplot as plt

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

logging.basicConfig()

from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

raw_data_path = Path().home() / "caiman_data"
movie_path = raw_data_path / 'animal_01' / "session_01" / 'save_gui.zarr'
# moviepath


2024-09-05 21:09:31.075440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:09:31.075471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:09:31.075846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## (optional): View hardware information

In [2]:
# !pip install cloudmesh-cmd5
!cms help # dont forget to call it after the install as it sets some defaults
!cms sysinfo

*** No help on # dont forget to call it after the install as it sets some defaults
+------------------+----------------------------------------------------------------------------------------------+
| Attribute        | Value                                                                                        |
+------------------+----------------------------------------------------------------------------------------------+
| cpu              |                                                                                              |
| cpu_cores        | 24                                                                                           |
| cpu_count        | 32                                                                                           |
| cpu_threads      | 32                                                                                           |
| date             | 2024-07-14 09:27:51.602620                                                          

## User input: input data path and plane number

the same path as [pre_processing](./pre_processing.ipynb)
parent_dir = Path().home() / 'caiman_data' / 'animal_01' / 'session_01'

In [4]:
mc.set_parent_raw_data_path(raw_data_path)

PosixPath('/home/mbo/caiman_data')

In [2]:
batch_path = raw_data_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(raw_data_path))

PosixPath('/home/mbo/caiman_data')

In [17]:
# create a new batch
# try:
# to load existing batches use `load_batch()`
    # df = mc.load_batch(batch_path)
# except (IsADirectoryError, FileNotFoundError):
df = mc.create_batch(batch_path, remove_existing=True)


df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


In [19]:
df = df.caiman.reload_from_disk()
if df.empty:

    mcorr_params = {'main':  # this key is necessary for specifying that these are the "main" params for the algorithm
        {
            'var_name_hdf5': 'plane_1',
            'max_shifts': [24, 24],
            'strides': [48, 48],
            'overlaps': [24, 24],
            'max_deviation_rigid': 3,
            'border_nan': 'copy',
            'pw_rigid': True,
            'gSig_filt': None
        },
    }

    df.caiman.add_item(
        algo='mcorr',
        input_movie_path=movie_path,
        params=mcorr_params,
        item_name=movie_path.stem,  # filename of the movie, but can be anything
    )

In [20]:
df.iloc[0].caiman.run()

2024-09-05 21:23:38.447801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:23:38.447830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:23:38.448201: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
plane_1
plane_1


In [18]:
import napari
napari.view_image(df.iloc[0].mcorr.get_output())
napari.current_viewer().add_image(zarr.open(movie_path)['plane_1'])

<Image layer 'Image [1]' at 0x7ff0a84a50d0>

In [ ]:
corr, pnr = correlation_pnr(mcorr_movie, swap_dim=False)

# Registration Grid Search

More runs with varying parameters, stored on disk in the dataframe batch.pickle

In [11]:
# copy the mcorr_params2 dict to make some changes
# dont accidentally run this twice (making a huge dataframe)
ran = False
if not ran:
  from copy import deepcopy

  new_params = deepcopy(mcorr_params)
  for plane in ['plane_1','plane_2']:
    # some variants of max_shifts
    for shifts in [2,64]: 
        for strides in [12,128]:
            overlaps = int(strides / 2)
            # deep copy is the safest way to copy dicts
            new_params = deepcopy(new_params)

            # assign the "max_shifts"
            new_params["main"]["max_shifts"] = (shifts, shifts)
            new_params["main"]["strides"] = (strides, strides)
            new_params["main"]["overlaps"] = (overlaps, overlaps)
            new_params['main']['var_name_hdf5'] = plane,
            df.caiman.add_item(
              algo='mcorr',
              item_name=plane,
              input_movie_path=movie_path,
              params=new_params
            )
            
df.caiman.reload_from_disk()

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,plane_1,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_1',), 'max_...",None,2024-09-05T21:17:15,None,None,None,e5c7dd26-9583-43da-93c6-74b317115886
1,mcorr,plane_1,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_1',), 'max_...",None,2024-09-05T21:17:15,None,None,None,1886de10-9f55-4662-9641-9b016d5b5c84
2,mcorr,plane_1,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_1',), 'max_...",None,2024-09-05T21:17:15,None,None,None,022bd6d4-9b25-4351-9d98-487db3d49cc6
3,mcorr,plane_1,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_1',), 'max_...",None,2024-09-05T21:17:15,None,None,None,2827c948-0a9a-431a-8dad-c16eaefadb85
4,mcorr,plane_2,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_2',), 'max_...",None,2024-09-05T21:17:15,None,None,None,a8450478-50f6-4d1e-bea8-e0a7d1b6b505
5,mcorr,plane_2,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_2',), 'max_...",None,2024-09-05T21:17:15,None,None,None,9a70bcb1-eb39-4218-bc97-21a13419167a
6,mcorr,plane_2,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_2',), 'max_...",None,2024-09-05T21:17:15,None,None,None,d78fe75c-b837-4872-a7f2-576878bf2b95
7,mcorr,plane_2,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': ('plane_2',), 'max_...",None,2024-09-05T21:17:15,None,None,None,816653c1-ecd6-49a1-b57e-3d3a85ce4924


In [9]:
for i, row in df.iterrows():
    if i in range(0, 86):
        df.caiman.remove_item(row.uuid)

In [10]:
df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


In [12]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

2024-09-05 21:17:48.122342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:17:48.122379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:17:48.122796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
mc failed, stored traceback in output


2024-09-05 21:17:51.441055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:17:51.441086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:17:51.441541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
mc failed, stored traceback in output


2024-09-05 21:17:54.755268: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:17:54.755302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:17:54.755687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

starting mc
mc failed, stored traceback in output


2024-09-05 21:18:08.253514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:18:08.253549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:18:08.253936: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
mc failed, stored traceback in output


2024-09-05 21:18:11.603943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 21:18:11.603977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 21:18:11.604384: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
mc failed, stored traceback in output


In [16]:
df = df.caiman.reload_from_disk()
print(df.iloc[0].outputs)

{'success': False, 'traceback': 'Traceback (most recent call last):\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/mesmerize_core/algorithms/mcorr.py", line 68, in run_algo\n    mc.motion_correct(save_movie=True)\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/caiman/motion_correction.py", line 234, in motion_correct\n    mi = min(mi, next(iterator).min()[()])\n                 ^^^^^^^^^^^^^^\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/caiman/base/movies.py", line 1901, in load_iter\n    for y in Y:\nTypeError: \'NoneType\' object is not iterable\n'}


In [43]:
# Now use more physiologically relevant parameter values

pix_res = 1

mx = 10/pix_res             
max_shifts = (int(mx), int(mx))       # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
max_deviation_rigid = 3     # maximum deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing rigid or piecewise rigid motion correction
shifts_opencv = True        # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used)
border_nan = 'copy'  # replicate values along the boundary (if True, fill in with NaN)
max_shifts

(10, 10)

In [44]:
# Set parameters for rigid motion correction
mcorr_params_2 =\
{ 'main':
    {
        'var_name_hdf5': 'plane_1',
        'max_shifts': max_shifts,   # max shift in pixels
        'strides': (48, 48),        # create a new patch every x pixels for pw-rigid correction
                                    # NOTE: "stride" for cnmf, "strides" for mcorr
        'overlaps': (24, 24),       # overlap between patches (size of patch strides+overlaps)
        'max_deviation_rigid': 3,   # maximum deviation allowed for patch with respect to rigid shifts
        'pw_rigid': True,          # flag for performing rigid or piecewise rigid motion correction
        'shifts_opencv': True,      # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used)
        'border_nan': 'copy',       # replicate values along the boundary (if True, fill in with NaN)
        'nonneg_movie': False,
    }
}

In [48]:
if df.empty:
        # add other param variant to the batch
    df.caiman.add_item(
    algo='mcorr',
    item_name='plane_1',
    input_movie_path=parent_dir,
    params=mcorr_params
    )

df = df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,plane_1,save_gui,"{'main': {'var_name_hdf5': 'plane_1', 'max_shi...","{'success': False, 'traceback': 'Traceback (mo...",2024-09-05T19:30:10,2024-09-05T19:30:35,0.11 sec,None,30b14eb5-9a25-401f-8fda-f89037c58c50


In [46]:
df.iloc[0].caiman.run()

2024-09-05 19:30:34.654339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 19:30:34.654368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 19:30:34.654752: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


starting mc
test

/home/mbo/caiman_data/animal_01/session_01/save_gui
mc failed, stored traceback in output


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman


<Popen: returncode: 0 args: '/home/mbo/caiman_data/animal_01/session_01/save...>

In [49]:
df.iloc[0].outputs['traceback']

'Traceback (most recent call last):\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/mesmerize_core/algorithms/mcorr.py", line 68, in run_algo\n    mc.motion_correct(save_movie=True)\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/caiman/motion_correction.py", line 234, in motion_correct\n    mi = min(mi, next(iterator).min()[()])\n                 ^^^^^^^^^^^^^^\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/caiman/base/movies.py", line 1916, in load_iter\n    for y in load(file_name, var_name_hdf5=var_name_hdf5,\n             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/caiman/base/movies.py", line 1521, in load\n    raise Exception(\'Unknown file type\')\nException: Unknown file type\n'

### View rigid template

In [ ]:
# load motion corrected movie
m_rig = cm.load(mc_rigid.mmap_file)
bord_px_rig = np.ceil(np.max(mc_rigid.shifts_rig)).astype(int)
##%% visualize templates
plt.figure(figsize = (20,10))
plt.imshow(mc_rigid.total_template_rig, cmap = 'gray');

Rigid-corrected movie

In [ ]:
##%% inspect movie
m_rig.resize(1, 1, downsample_ratio).play(
    q_max=99.5, fr=30, magnification=2, bord_px = 0*bord_px_rig) # press q to exit

Rigid Template shifts

In [ ]:
##%% plot rigid shifts
plt.close()
plt.figure(figsize = (20,10))
plt.plot(mc_rigid.shifts_rig)
plt.legend(['x shifts','y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels');

In [ ]:
# correct for rigid motion correction and save the file (in memory mapped form)
mc.motion_correct(save_movie=True)


## Piecewise rigid registration

While rigid registration corrected for a lot of the movement, there is still non-uniform motion present in the registered file.

- To correct for that we can use piece-wise rigid registration directly in the original file by setting mc.pw_rigid=True.
- As before the registered file is saved in a memory mapped format in the location given by mc.mmap_file.


In [ ]:
%%capture
##%% motion correct piecewise rigid
mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
mc.template = mc.mmap_file  # use the template obtained before to save in computation (optional)

mc.motion_correct(save_movie=True, template=mc.total_template_rig)
m_els = cm.load(mc.fname_tot_els)
m_els.resize(1, 1, downsample_ratio).play(
    q_max=99.5, fr=30, magnification=2,bord_px = bord_px_rig)

## Correlation metrics

Create a couple of summary images of the movie, including:
- maximum projection (the maximum value of each pixel) 
- correlation image (how correlated each pixel is with its neighbors)

If a pixel comes from an active neural component it will tend to be highly correlated with its neighbors.

In [ ]:
plt.close()
plt.figure(figsize = (20,10))
plt.subplot(2, 1, 1)
plt.plot(mc.x_shifts_els)
plt.ylabel('x shifts (pixels)')
plt.subplot(2, 1, 2)
plt.plot(mc.y_shifts_els)
plt.ylabel('y_shifts (pixels)')
plt.xlabel('frames')
##%% compute borders to exclude
bord_px_els = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                 np.max(np.abs(mc.y_shifts_els)))).astype(int)

## Motion Corretion: Optical Flow

In [ ]:
##%% plot the results of Residual Optical Flow
fls = [cm.paths.fname_derived_presuffix(mc.fname_tot_els[0], 'metrics', swapsuffix='npz'),
       cm.paths.fname_derived_presuffix(mc.fname_tot_rig[0], 'metrics', swapsuffix='npz'),
       cm.paths.fname_derived_presuffix(mc.fname[0],         'metrics', swapsuffix='npz'),
      ]

plt.figure(figsize = (20,10))
for cnt, fl, metr in zip(range(len(fls)), fls, ['pw_rigid','rigid','raw']):
    with np.load(fl) as ld:
        print(ld.keys())
        print(fl)
        print(str(np.mean(ld['norms'])) + '+/-' + str(np.std(ld['norms'])) +
              ' ; ' + str(ld['smoothness']) + ' ; ' + str(ld['smoothness_corr']))
        
        plt.subplot(len(fls), 3, 1 + 3 * cnt)
        plt.ylabel(metr)
        print(f"Loading data with base {fl[:-12]}")
        try:
            mean_img = np.mean(
            cm.load(fl[:-12] + '.mmap'), 0)[12:-12, 12:-12]
        except:
            try:
                mean_img = np.mean(
                    cm.load(fl[:-12] + '.tif'), 0)[12:-12, 12:-12]
            except:
                mean_img = np.mean(
                    cm.load(fl[:-12] + 'hdf5'), 0)[12:-12, 12:-12]
                    
        lq, hq = np.nanpercentile(mean_img, [.5, 99.5])
        plt.imshow(mean_img, vmin=lq, vmax=hq)
        plt.title('Mean')
        plt.subplot(len(fls), 3, 3 * cnt + 2)
        plt.imshow(ld['img_corr'], vmin=0, vmax=.35)
        plt.title('Corr image')
        plt.subplot(len(fls), 3, 3 * cnt + 3)
        flows = ld['flows']
        plt.imshow(np.mean(
        np.sqrt(flows[:, :, :, 0]**2 + flows[:, :, :, 1]**2), 0), vmin=0, vmax=0.3)
        plt.colorbar()
        plt.title('Mean optical flow');  

# Cleanup

Make sure our parallel cluster is shut down.

In [ ]:
if 'dview' in locals():
    cm.stop_server(dview=dview)
elif 'cluster' in locals():
    cm.stop_server(dview=cluster)